### Test random environment with OpenAI gym

In [1]:
import gym
import random

In [15]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [16]:
states

4

In [17]:
actions

2

In [18]:
episodes = 10
# gym.make(env, render_mode='human')


for episode in range(1, episodes + 1) :
    state = env.reset()
    done = False
    score = 0
    
    while not done:
#         env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    
    print ('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:24.0
Episode:2 Score:28.0
Episode:3 Score:11.0
Episode:4 Score:25.0
Episode:5 Score:33.0
Episode:6 Score:13.0
Episode:7 Score:20.0
Episode:8 Score:37.0
Episode:9 Score:18.0
Episode:10 Score:26.0


### Deep Learning Model with Keras


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow

In [3]:
def build_model(states, actions) :
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [21]:
model = build_model(states, actions)

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


### Build Agent with Keras-RL

In [4]:
from rl.agents import DQNAgent
from rl.policy import *
from rl.memory import SequentialMemory

In [5]:
def build_agent(model, actions) :
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, nb_actions= actions, nb_steps_warmup = 10, 
                   target_model_update=1e-2)
    return dqn

In [25]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 7:00 - reward: 1.0000

c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   59/10000 [..............................] - ETA: 1:31 - reward: 1.0000

c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\users\dhruv\appdata\l

10000/10000 [==============================] - 41s 4ms/step - reward: 1.0000
95 episodes - episode_reward: 104.621 [10.000, 359.000] - loss: 2.205 - mae: 19.298 - mean_q: 39.163

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: 1.0000
45 episodes - episode_reward: 221.778 [181.000, 295.000] - loss: 3.335 - mae: 40.126 - mean_q: 81.147

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: 1.0000
36 episodes - episode_reward: 274.139 [194.000, 494.000] - loss: 2.759 - mae: 42.925 - mean_q: 86.493

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: 1.0000
39 episodes - episode_reward: 260.795 [169.000, 500.000] - loss: 2.199 - mae: 43.148 - mean_q: 86.865

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
done, took 208.879 seconds


In [26]:
scores =  dqn.test(env, nb_episodes = 100, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 230.000, steps: 230
Episode 2: reward: 222.000, steps: 222
Episode 3: reward: 292.000, steps: 292
Episode 4: reward: 246.000, steps: 246
Episode 5: reward: 286.000, steps: 286
Episode 6: reward: 190.000, steps: 190
Episode 7: reward: 206.000, steps: 206
Episode 8: reward: 189.000, steps: 189
Episode 9: reward: 232.000, steps: 232
Episode 10: reward: 206.000, steps: 206
Episode 11: reward: 232.000, steps: 232
Episode 12: reward: 269.000, steps: 269
Episode 13: reward: 252.000, steps: 252
Episode 14: reward: 259.000, steps: 259
Episode 15: reward: 233.000, steps: 233
Episode 16: reward: 250.000, steps: 250
Episode 17: reward: 224.000, steps: 224
Episode 18: reward: 237.000, steps: 237
Episode 19: reward: 208.000, steps: 208
Episode 20: reward: 183.000, steps: 183
Episode 21: reward: 263.000, steps: 263
Episode 22: reward: 187.000, steps: 187
Episode 23: reward: 190.000, steps: 190
Episode 24: reward: 252.000, steps: 252
Episode 25: reward: 

### Reloading model from memory

In [27]:
dqn.save_weights('dqn_weights.h5f', overwrite = True)

In [28]:
del model
del dqn
del env

In [6]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])

c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
dqn.load_weights('dqn_weights.h5f')

In [8]:
_ = dqn.test(env, nb_episodes = 5, visualize= True)

Testing for 5 episodes ...


c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\users\dhruv\appdata\local\programs\python\python39\lib\site-packages\gym\core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: reward: 237.000, steps: 237
Episode 2: reward: 242.000, steps: 242
Episode 3: reward: 222.000, steps: 222
Episode 4: reward: 227.000, steps: 227
Episode 5: reward: 336.000, steps: 336
